In [ ]:
!wget https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install findspark

In [ ]:
!wget "https://raw.githubusercontent.com/AISCIENCES/course-master-big-data-with-pyspark-and-aws/main/Code/06-ETL/WordData.txt"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").getOrCreate()

In [ ]:
from pyspark.sql.functions import col, lit, explode
import pyspark.sql.functions as f

# **Extract**

In [ ]:
df = spark.read.text("/content/WordData.txt")
df.show(5)

+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
+--------------------+
only showing top 5 rows



# **Transform**

In [ ]:
wordsDF = df.withColumn("splitted_data", f.split("value", " ")).withColumn("words", explode(col("splitted_data"))).select(col("words"))
wordsDF.cache()
wordsDF.show()

+--------+
|   words|
+--------+
|    This|
|      is|
|       a|
|Japanese|
|    doll|
|     The|
|    team|
| members|
|    were|
|    hard|
|      to|
|    tell|
|   apart|
|   since|
|    they|
|     all|
|    wore|
|   their|
|    hair|
|      in|
+--------+
only showing top 20 rows



Word Count

In [ ]:
wordCountDF = wordsDF.groupBy("words").count()
wordCountDF.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



# **Load**

In [ ]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://database-1.c0sanhk4hmut.us-west-2.rds.amazonaws.com/"
table = "WordCount"
user = "postgres"
password = ""

wordCount.write.format("jdbc").option("driver", driver).option("url",url).option("dbtable", table).option("mode", "append").option("user",user).option("password", password).save()